<a href="https://colab.research.google.com/github/Fayli775/INFOSYS722-Iteration4/blob/main/03_DP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
# !pip install pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T
from pyspark.sql.functions import col, when, lit, to_timestamp, to_date, date_format, year, month, dayofmonth, hour, dayofweek, format_string, regexp_replace
from google.colab import drive

drive.mount('/content/drive')

spark = (
    SparkSession.builder
    .appName("Iteration4-DataPreparation")
    .config("spark.sql.adaptive.enabled", "true")
    .getOrCreate()
)

traffic_fp = "/content/drive/MyDrive/722/TMS_Telemetry_Sites.csv"
holidays_fp = "/content/drive/MyDrive/722/NZ_Holidays.csv"

df_raw = spark.read.csv(traffic_fp, header=True, inferSchema=True)

def null_report(df, title=""):
    nr = df.select([F.count(F.when(F.col(c).isNull() | F.isnan(c), c)).alias(c) for c in df.columns])
    print(f"[NULL REPORT] {title} rows={df.count()} cols={len(df.columns)}")
    nr.show(truncate=False)

def count_log(df, tag):
    cnt = df.count()
    print(f"[COUNT] {tag} -> {cnt}")
    return cnt

# Work with original column names - no renaming
df = df_raw

count_log(df, "raw")
df.printSchema()

# Use original column names for processing
target_col = "Traffic Count"

# Drop columns using original names
drop_text = [c for c in ["Site Alias", "Site Description"] if c in df.columns]
drop_sys = [c for c in ["OBJECTID"] if c in df.columns]
df = df.drop(*drop_text).drop(*drop_sys)

# Cast target column to double
if target_col in df.columns:
    df = df.withColumn(target_col, col(target_col).cast("double"))

# Filter for Auckland using original column name
if "Region Name" in df.columns:
    df = df.filter(col("Region Name").contains("Auckland"))

count_log(df, "after selection and region filter")
null_report(df.select([c for c in df.columns if c in ["Start Date","Lane Number","Flow Direction","Class Weight",target_col]]), "selection key fields")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[COUNT] raw -> 6660911
root
 |-- OBJECTID: integer (nullable = true)
 |-- Start Date: string (nullable = true)
 |-- Site Alias: integer (nullable = true)
 |-- Region Name: string (nullable = true)
 |-- Site Reference: string (nullable = true)
 |-- Class Weight: string (nullable = true)
 |-- Site Description: string (nullable = true)
 |-- Lane Number: integer (nullable = true)
 |-- Flow Direction: integer (nullable = true)
 |-- Traffic Count: double (nullable = true)

[COUNT] after selection and region filter -> 1922734
[NULL REPORT] selection key fields rows=1922734 cols=5
+----------+------------+-----------+--------------+-------------+
|Start Date|Class Weight|Lane Number|Flow Direction|Traffic Count|
+----------+------------+-----------+--------------+-------------+
|0         |0           |0          |0             |0            |
+----------+-----------

In [42]:
print("Sample of filtered data:")
df.show(10, truncate=False)

Sample of filtered data:
+--------------------+-------------+--------------+------------+-----------+--------------+-------------+
|Start Date          |Region Name  |Site Reference|Class Weight|Lane Number|Flow Direction|Traffic Count|
+--------------------+-------------+--------------+------------+-----------+--------------+-------------+
|4/5/2018 12:00:00 AM|02 - Auckland|01600023      |Light       |2          |2             |18585.0      |
|4/5/2018 12:00:00 AM|02 - Auckland|01600023      |Heavy       |1          |1             |924.0        |
|4/5/2018 12:00:00 AM|02 - Auckland|01600023      |Light       |1          |1             |18508.0      |
|4/5/2018 12:00:00 AM|02 - Auckland|01600023      |Heavy       |2          |2             |930.0        |
|4/6/2018 12:00:00 AM|02 - Auckland|01600023      |Heavy       |2          |2             |938.0        |
|4/6/2018 12:00:00 AM|02 - Auckland|01600023      |Light       |1          |1             |18775.5      |
|4/6/2018 12:00:00 AM

In [43]:
if target_col in df.columns:
    df = df.filter(col(target_col).isNotNull())

if "Lane Number" in df.columns:
    df = df.withColumn("Lane Number", col("Lane Number").cast("int"))
    df = df.withColumn(
        "lane_number_fixed",
        when(col("Lane Number") < 1, lit(1)).when(col("Lane Number") > 6, lit(6)).otherwise(col("Lane Number"))
    ).withColumn(
        "lane_number_out_of_range",
        when((col("Lane Number") < 1) | (col("Lane Number") > 6), lit(1)).otherwise(lit(0))
    ).drop("Lane Number").withColumnRenamed("lane_number_fixed", "Lane Number")

if target_col in df.columns:
    try:
        p999 = df.approxQuantile(target_col, [0.999], 0.001)[0]
    except Exception:
        p999 = 60000.0
    soft_cap = float(p999 if p999 and p999 > 0 else 60000.0)
    hard_cap = 200000.0
    print(f"[INFO] traffic_count soft_cap={soft_cap:.1f}, hard_cap={hard_cap:.1f}")
    df = df.withColumn("traffic_count_raw", col(target_col)).withColumn(target_col, F.least(col(target_col), lit(soft_cap), lit(hard_cap)))

if "Site Reference" in df.columns:
    completeness = df.filter(col("Site Reference").isNotNull()).count() / max(df.count(), 1)
    print(f"[INFO] site_reference completeness: {completeness:.4f}")
    df = df.drop("Site Reference")

# Enhanced final data quality assessment
total_records = df.count()
print(f"Total records after cleaning: {total_records:,}")

# Check all key analytical fields
key_fields = ["Lane Number", "Flow Direction", "Class Weight", target_col]
available_fields = [c for c in key_fields if c in df.columns]

print(f"\nAnalytical fields assessment:")
for field in available_fields:
    null_count = df.filter(col(field).isNull()).count()
    completeness = ((total_records - null_count) / total_records) * 100
    print(f"  {field}: {completeness:.2f}% complete ({null_count:,} nulls)")

# Overall null report for all key fields
null_report(df.select(available_fields), "FINAL CLEANED DATASET - Key Analytical Fields")

# Additional statistics
print(f"\nDataset summary:")
print(f"  - Total columns: {len(df.columns)}")
print(f"  - Key analytical fields: {len(available_fields)}")
print(f"  - Records retained: {total_records:,}")

# Lane number corrections summary (if applied)
if "lane_number_out_of_range" in df.columns:
    corrections = df.filter(col("lane_number_out_of_range") == 1).count()
    print(f"\nLane number corrections applied: {corrections:,} records")
    df = df.drop("lane_number_out_of_range")  # Clean up audit column

# Traffic count capping summary
if "traffic_count_raw" in df.columns:
    capped_records = df.filter(col("traffic_count_raw") != col(target_col)).count()
    print(f"Traffic count values capped: {capped_records:,} records")

    if capped_records > 0:
        print("Traffic count distribution after capping:")
        df.select(target_col).describe().show()

[INFO] traffic_count soft_cap=33898.5, hard_cap=200000.0
[INFO] site_reference completeness: 1.0000
Total records after cleaning: 1,922,734

Analytical fields assessment:
  Lane Number: 100.00% complete (0 nulls)
  Flow Direction: 100.00% complete (0 nulls)
  Class Weight: 100.00% complete (0 nulls)
  Traffic Count: 100.00% complete (0 nulls)
[NULL REPORT] FINAL CLEANED DATASET - Key Analytical Fields rows=1922734 cols=4
+-----------+--------------+------------+-------------+
|Lane Number|Flow Direction|Class Weight|Traffic Count|
+-----------+--------------+------------+-------------+
|0          |0             |0           |0            |
+-----------+--------------+------------+-------------+


Dataset summary:
  - Total columns: 8
  - Key analytical fields: 4
  - Records retained: 1,922,734

Lane number corrections applied: 0 records
Traffic count values capped: 0 records


In [44]:
print("Sample of data:")
df.show(10, truncate=False)

Sample of data:
+--------------------+-------------+------------+--------------+-------------+-----------+-----------------+
|Start Date          |Region Name  |Class Weight|Flow Direction|Traffic Count|Lane Number|traffic_count_raw|
+--------------------+-------------+------------+--------------+-------------+-----------+-----------------+
|4/5/2018 12:00:00 AM|02 - Auckland|Light       |2             |18585.0      |2          |18585.0          |
|4/5/2018 12:00:00 AM|02 - Auckland|Heavy       |1             |924.0        |1          |924.0            |
|4/5/2018 12:00:00 AM|02 - Auckland|Light       |1             |18508.0      |1          |18508.0          |
|4/5/2018 12:00:00 AM|02 - Auckland|Heavy       |2             |930.0        |2          |930.0            |
|4/6/2018 12:00:00 AM|02 - Auckland|Heavy       |2             |938.0        |2          |938.0            |
|4/6/2018 12:00:00 AM|02 - Auckland|Light       |1             |18775.5      |1          |18775.5          |
|4/

In [45]:
# 3.3 Data Construction
date_col = "Start Date"
if date_col in df.columns:
    df = df.withColumn("StartDate_parsed", F.to_timestamp(F.col(date_col), "M/d/yyyy h:mm:ss a")) \
           .withColumn("year", F.year("StartDate_parsed")) \
           .withColumn("month", F.month("StartDate_parsed")) \
           .withColumn("day", F.dayofmonth("StartDate_parsed")) \
           .withColumn("hour", F.hour("StartDate_parsed")) \
           .withColumn("dow_num", F.dayofweek("StartDate_parsed")) \
           .withColumn("weekday", F.date_format("StartDate_parsed", "E"))

    # Check parsing success
    parsing_success = df.filter(col("StartDate_parsed").isNotNull()).count()
    total_rows = df.count()
    print(f"[INFO] Date parsing success: {parsing_success}/{total_rows} ({parsing_success/total_rows*100:.1f}%)")

if "Flow Direction" in df.columns:
    df = df.withColumn("Flow Direction", col("Flow Direction").cast("int"))
    df = df.withColumn(
        "flow_direction_grp",
        when(col("Flow Direction").isin(1, 2), col("Flow Direction").cast("string"))
        .when(col("Flow Direction").isin(3, 4, 5, 6), lit("Other"))
        .otherwise(lit("Other"))
    ).drop("Flow Direction").withColumnRenamed("flow_direction_grp", "Flow Direction")

if "Class Weight" in df.columns:
    df = df.withColumn("Class Weight", F.coalesce(col("Class Weight").cast("string"), lit("Unknown")))

# Update the essential columns list to match the new column names
essential_cols = [c for c in ["year","month","day","hour","weekday","Lane Number","Flow Direction","Class Weight",target_col] if c in df.columns]
null_report(df.select(*essential_cols), "after construction essential")

[INFO] Date parsing success: 1922734/1922734 (100.0%)
[NULL REPORT] after construction essential rows=1922734 cols=9
+----+-----+---+----+-------+-----------+--------------+------------+-------------+
|year|month|day|hour|weekday|Lane Number|Flow Direction|Class Weight|Traffic Count|
+----+-----+---+----+-------+-----------+--------------+------------+-------------+
|0   |0    |0  |0   |0      |0          |0             |0           |0            |
+----+-----+---+----+-------+-----------+--------------+------------+-------------+



In [53]:
# 3.3.5 Feature Construction Validation

# Check all essential constructed features
essential_cols = [c for c in ["year","month","day","hour","weekday","Lane Number","Flow Direction","Class Weight",target_col] if c in df.columns]
null_report(df.select(*essential_cols), "after construction essential")

# Temporal feature range validation
print("\nTemporal feature ranges:")
if "year" in df.columns:
    year_range = df.select(F.min("year"), F.max("year")).collect()[0]
    print(f"  Year range: {year_range[0]} to {year_range[1]}")

if "month" in df.columns:
    month_range = df.select(F.min("month"), F.max("month")).collect()[0]
    print(f"  Month range: {month_range[0]} to {month_range[1]}")

if "hour" in df.columns:
    hour_stats = df.select("hour").distinct().orderBy("hour").collect()
    hours = [row[0] for row in hour_stats]
    print(f"  Hour values: {hours}")

# Flow Direction distribution after grouping
print("\nFlow Direction category distribution:")
if "Flow Direction" in df.columns:
    flow_dist = df.groupBy("Flow Direction").count().orderBy(F.desc("count"))
    flow_dist.show()

# Class Weight distribution
print("Class Weight distribution:")
if "Class Weight" in df.columns:
    class_dist = df.groupBy("Class Weight").count().orderBy(F.desc("count"))
    class_dist.show()

# Weekday distribution validation
print("Weekday distribution:")
if "weekday" in df.columns:
    weekday_dist = df.groupBy("weekday").count().orderBy("weekday")
    weekday_dist.show()

# Sample of constructed features
print("\nSample of constructed data with new features:")
sample_cols = ["Start Date"] + essential_cols
available_sample_cols = [c for c in sample_cols if c in df.columns]
df.select(*available_sample_cols).show(5, truncate=False)

# Construction impact summary
total_original_cols = len([c for c in df.columns if not c.startswith("year") and not c.startswith("month") and not c.startswith("day") and not c.startswith("hour") and not c.startswith("weekday") and not c.startswith("dow_num") and c != "StartDate_parsed"])
total_current_cols = len(df.columns)
constructed_features = len([c for c in ["year","month","day","hour","weekday","dow_num","StartDate_parsed"] if c in df.columns])

print(f"\nConstruction Summary:")
print(f"  - Original analytical features: {len(essential_cols) - constructed_features}")
print(f"  - Newly constructed features: {constructed_features}")
print(f"  - Total analytical features: {len(essential_cols)}")
print(f"  - Records maintained: {df.count():,}")

[NULL REPORT] after construction essential rows=1922734 cols=9
+----+-----+---+----+-------+-----------+--------------+------------+-------------+
|year|month|day|hour|weekday|Lane Number|Flow Direction|Class Weight|Traffic Count|
+----+-----+---+----+-------+-----------+--------------+------------+-------------+
|0   |0    |0  |0   |0      |0          |0             |0           |0            |
+----+-----+---+----+-------+-----------+--------------+------------+-------------+


Temporal feature ranges:
  Year range: 2017 to 2025
  Month range: 1 to 12
  Hour values: [0, 12]

Flow Direction category distribution:
+--------------+------+
|Flow Direction| count|
+--------------+------+
|             1|649176|
|             2|643475|
|         Other|630083|
+--------------+------+

Class Weight distribution:
+------------+-------+
|Class Weight|  count|
+------------+-------+
|       Light|1216337|
|       Heavy| 706397|
+------------+-------+

Weekday distribution:
+-------+------+
|wee

In [46]:
print("Sample of data:")
df.show(10, truncate=False)

Sample of data:
+--------------------+-------------+------------+-------------+-----------+-----------------+-------------------+----+-----+---+----+-------+-------+--------------+
|Start Date          |Region Name  |Class Weight|Traffic Count|Lane Number|traffic_count_raw|StartDate_parsed   |year|month|day|hour|dow_num|weekday|Flow Direction|
+--------------------+-------------+------------+-------------+-----------+-----------------+-------------------+----+-----+---+----+-------+-------+--------------+
|4/5/2018 12:00:00 AM|02 - Auckland|Light       |18585.0      |2          |18585.0          |2018-04-05 00:00:00|2018|4    |5  |0   |5      |Thu    |2             |
|4/5/2018 12:00:00 AM|02 - Auckland|Heavy       |924.0        |1          |924.0            |2018-04-05 00:00:00|2018|4    |5  |0   |5      |Thu    |1             |
|4/5/2018 12:00:00 AM|02 - Auckland|Light       |18508.0      |1          |18508.0          |2018-04-05 00:00:00|2018|4    |5  |0   |5      |Thu    |1         

In [47]:
from pyspark.sql.functions import col, lit, to_date, year, month, dayofmonth

# 1) Load and normalize holiday file
hol = spark.read.csv(holidays_fp, header=True, inferSchema=True)
print(f"[INFO] Holiday file columns: {hol.columns}")

# lowercase all column names to avoid case mismatches
for c in hol.columns:
    hol = hol.withColumnRenamed(c, c.lower())

if "date" not in hol.columns:
    raise ValueError("NZ_Holidays.csv missing 'date' column")

# 2) Build join keys and a single holiday flag column on the RIGHT side
hol = (
    hol.withColumn("date_only", to_date(col("date")))
       .withColumn("year",  year(col("date_only")))
       .withColumn("month", month(col("date_only")))
       .withColumn("day",   dayofmonth(col("date_only")))
       .select("year", "month", "day")
       .distinct()
       .withColumn("hol_is_holiday", lit(True))  # right-side flag with unique name
)

print(f"[INFO] Holiday unique dates: {hol.count()}")

# 3) Ensure main df has join keys
join_keys = ["year", "month", "day"]
missing_keys = [k for k in join_keys if k not in df.columns]
if missing_keys:
    raise ValueError(f"Missing join keys in main df: {missing_keys}. Make sure 3.3 created year/month/day.")

# 4) Drop any existing is_holiday in main df to avoid ambiguity (safe/idempotent)
if "is_holiday" in df.columns:
    df = df.drop("is_holiday")

# 5) Alias, left join, coalesce the right-side flag into a single 'is_holiday'
l = df.alias("l")
r = hol.alias("r")
df = (
    l.join(r, on=join_keys, how="left")
     .withColumn("is_holiday", F.coalesce(col("r.hol_is_holiday"), lit(False)))
     .drop("hol_is_holiday")  # drop the right-side helper column
)

# 6) Optional: cast to int for modelling compatibility
df = df.withColumn("is_holiday", col("is_holiday").cast("int"))

# 7) Sanity checks
print(f"[DEBUG] Final df columns: {df.columns}")
holiday_count = df.filter(col("is_holiday") == 1).count()
print(f"[INFO] Records marked as holidays: {holiday_count:,}")
null_report(df.select([c for c in ["year","month","day","is_holiday"] if c in df.columns]), "after holiday join")


[INFO] Holiday file columns: ['date', 'name']
[INFO] Holiday unique dates: 103
[DEBUG] Final df columns: ['year', 'month', 'day', 'Start Date', 'Region Name', 'Class Weight', 'Traffic Count', 'Lane Number', 'traffic_count_raw', 'StartDate_parsed', 'hour', 'dow_num', 'weekday', 'Flow Direction', 'is_holiday']
[INFO] Records marked as holidays: 60,557
[NULL REPORT] after holiday join rows=1922734 cols=4
+----+-----+---+----------+
|year|month|day|is_holiday|
+----+-----+---+----------+
|0   |0    |0  |0         |
+----+-----+---+----------+



In [48]:
print("Sample of data:")
df.show(10, truncate=False)

Sample of data:
+----+-----+---+--------------------+-------------+------------+-------------+-----------+-----------------+-------------------+----+-------+-------+--------------+----------+
|year|month|day|Start Date          |Region Name  |Class Weight|Traffic Count|Lane Number|traffic_count_raw|StartDate_parsed   |hour|dow_num|weekday|Flow Direction|is_holiday|
+----+-----+---+--------------------+-------------+------------+-------------+-----------+-----------------+-------------------+----+-------+-------+--------------+----------+
|2018|4    |5  |4/5/2018 12:00:00 AM|02 - Auckland|Light       |18585.0      |2          |18585.0          |2018-04-05 00:00:00|0   |5      |Thu    |2             |0         |
|2018|4    |5  |4/5/2018 12:00:00 AM|02 - Auckland|Heavy       |924.0        |1          |924.0            |2018-04-05 00:00:00|0   |5      |Thu    |1             |0         |
|2018|4    |5  |4/5/2018 12:00:00 AM|02 - Auckland|Light       |18508.0      |1          |18508.0       

In [54]:
# 3.5 Data Reformatting and Export


# helper to pick whichever variant exists
def pick(*cands):
    for c in cands:
        if c in df.columns:
            return c
    return None

# Final schema standardization - ensure consistent column selection
final_analytical_vars = [
    pick("year"),
    pick("month"),
    pick("day"),
    pick("hour"),
    pick("weekday"),
    pick("region_name", "Region Name"),
    pick("lane_number", "Lane Number"),
    pick("flow_direction", "Flow Direction"),
    pick("class_weight", "Class Weight"),
    pick("is_holiday"),
    pick("traffic_count", "Traffic Count")
]

# Add audit trail if available
raw_target = pick("traffic_count_raw")
if raw_target:
    final_analytical_vars.append(raw_target)

# Select final modeling dataset
final_cols = [c for c in final_analytical_vars if c]
df_final = df.select(*final_cols)

print(f"[SCHEMA] Final dataset structure: {len(final_cols)} variables selected")
print(f"[SCHEMA] Variables: {[c for c in final_cols]}")

# Final data export in multiple formats
out_parquet = "/content/drive/MyDrive/722/output/03_prepared.parquet"
out_csv = "/content/drive/MyDrive/722/output/03_prepared.csv"

print(f"[EXPORT] Saving to Parquet format...")
df_final.repartition(1).write.mode("overwrite").parquet(out_parquet)

print(f"[EXPORT] Saving to CSV format...")
df_final.repartition(1).write.mode("overwrite").option("header", True).csv(out_csv)

print(f"[SAVED] {out_parquet}")
print(f"[SAVED] {out_csv}")

# Final dataset characterization
count_log(df_final, "prepared dataset")
print(f"[FINAL] Ready for modeling: {df_final.count():,} records, {len(df_final.columns)} variables")

# 3.6 Dataset Summary & Validation

# resolve column names for summary
target_c  = pick("traffic_count", "Traffic Count")
flow_c    = pick("flow_direction", "Flow Direction")
class_c   = pick("class_weight", "Class Weight")
weekday_c = pick("weekday")
holiday_c = pick("is_holiday")
raw_c     = pick("traffic_count_raw")

# Categorical variable distributions
print("\n[CATEGORICAL DISTRIBUTIONS]")
for var_name, col_name in [("Flow Direction", flow_c), ("Class Weight", class_c), ("Weekday", weekday_c), ("Holiday Status", holiday_c)]:
    if col_name:
        print(f"\n{var_name}:")
        df_final.groupBy(col_name).count().orderBy(F.desc("count")).show(10, truncate=False)

# Target variable descriptive statistics
if target_c:
    print(f"\n[TARGET VARIABLE STATISTICS] - {target_c}")
    stats_df = df_final.select(
        F.mean(col(target_c)).alias("mean"),
        F.percentile_approx(col(target_c), 0.5).alias("median"),
        F.min(col(target_c)).alias("min"),
        F.max(col(target_c)).alias("max"),
        F.stddev(col(target_c)).alias("std")
    )
    stats_df.show(truncate=False)

# Data preparation impact summary
print("\n[DATA PREPARATION IMPACT SUMMARY]")

if holiday_c:
    holiday_pct = df_final.agg(F.mean(col(holiday_c).cast("double"))).collect()[0][0]
    print(f"  Holiday coverage: {holiday_pct:.2%} of records")

if raw_c and target_c:
    capped_pct = df_final.filter(col(raw_c) != col(target_c)).count() / df_final.count()
    print(f"  Traffic count capping applied: {capped_pct:.4%} of records")

print(f"  Final record count: {df_final.count():,}")
print(f"  Final variable count: {len(df_final.columns)}")
print(f"  Data completeness: 100% (all preparation stages successful)")


[SCHEMA] Final dataset structure: 12 variables selected
[SCHEMA] Variables: ['year', 'month', 'day', 'hour', 'weekday', 'Region Name', 'Lane Number', 'Flow Direction', 'Class Weight', 'is_holiday', 'Traffic Count', 'traffic_count_raw']
[EXPORT] Saving to Parquet format...
[EXPORT] Saving to CSV format...
[SAVED] /content/drive/MyDrive/722/output/03_prepared.parquet
[SAVED] /content/drive/MyDrive/722/output/03_prepared.csv
[COUNT] prepared dataset -> 1922734
[FINAL] Ready for modeling: 1,922,734 records, 12 variables

[CATEGORICAL DISTRIBUTIONS]

Flow Direction:
+--------------+------+
|Flow Direction|count |
+--------------+------+
|1             |649176|
|2             |643475|
|Other         |630083|
+--------------+------+


Class Weight:
+------------+-------+
|Class Weight|count  |
+------------+-------+
|Light       |1216337|
|Heavy       |706397 |
+------------+-------+


Weekday:
+-------+------+
|weekday|count |
+-------+------+
|Wed    |275744|
|Sat    |275395|
|Thu    |27515

In [52]:
print("Sample of data:")
df.show(10, truncate=False)

Sample of data:
+----+-----+---+-------------+------------+-------------+-----------+-----------------+----+-------+-------+--------------+----------+
|year|month|day|Region Name  |Class Weight|Traffic Count|Lane Number|traffic_count_raw|hour|dow_num|weekday|Flow Direction|is_holiday|
+----+-----+---+-------------+------------+-------------+-----------+-----------------+----+-------+-------+--------------+----------+
|2018|4    |5  |02 - Auckland|Light       |18585.0      |2          |18585.0          |0   |5      |Thu    |2             |0         |
|2018|4    |5  |02 - Auckland|Heavy       |924.0        |1          |924.0            |0   |5      |Thu    |1             |0         |
|2018|4    |5  |02 - Auckland|Light       |18508.0      |1          |18508.0          |0   |5      |Thu    |1             |0         |
|2018|4    |5  |02 - Auckland|Heavy       |930.0        |2          |930.0            |0   |5      |Thu    |2             |0         |
|2018|4    |6  |02 - Auckland|Heavy    